# 2019年度　パターン認識最終課題

パターン認識最終課題では、銀行の顧客に関するデータセットを使って、顧客の婚姻状況を予測するモデルを構築してもらいます。

学籍番号と氏名を記入してください。
記入例：1811000 先端　太郎

1911328 TAO XIANGKUN

## 課題１: 色々なモデルを試してみよう！
まずは、講義で扱った様々な識別器を試してみましょう。
以下に示すロジスティック回帰の例を参考に、指示に従いSVM、K-近傍法、決定木、ランダムフォレストの実装を行ってもらいます。

In [55]:
from IPython.display import display
import pandas as pd
from pandas import DataFrame, Series
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,precision_recall_fscore_support
import warnings
warnings.filterwarnings('ignore')

In [56]:
%matplotlib inline  
import matplotlib.pyplot as plt
import numpy as np
RAND_SEED = 42

### Step1 データの読み込み
まずは、データを読み込みます。

In [57]:
df_train_data = pd.read_csv('train.csv')

今回は、この中で、marital（婚姻状況）を予測するモデルの構築を行ってもらいます。

### Step 2　使用する列の抽出
次に、使用する列だけを取り出します。　今回はmaritalと、予測に使えそうな age, housing を取り出します。

In [58]:
y_ = df_train_data.marital# ラベルデータ
y_.value_counts()

married     12141
single       5605
divorced     2254
Name: marital, dtype: int64

In [59]:
DataFrame(y_).head(10)

,marital
0,married
1,married
2,divorced
3,married
4,married
5,single
6,married
7,single
8,divorced
9,married


ラベルデータの上位10件を表示しました。

In [60]:
X_ = df_train_data[['age', "job", "education", "default", "housing", "loan", "y"]]
df_train_data.head(5)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,58,housemaid,married,basic.4y,unknown,yes,no,cellular,aug,mon,...,2,999,0,nonexistent,1.4,93.444,-36.1,4.970,5228.1,no
1,38,technician,married,professional.course,no,yes,no,telephone,may,tue,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.856,5191.0,no
2,81,retired,divorced,basic.4y,no,yes,no,cellular,oct,thu,...,1,999,1,failure,-1.1,94.601,-49.5,0.987,4963.6,no
3,49,blue-collar,married,basic.6y,no,yes,no,telephone,may,fri,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.864,5191.0,yes
4,38,unemployed,married,basic.4y,unknown,no,no,telephone,jun,thu,...,1,999,0,nonexistent,1.4,94.465,-41.8,4.961,5228.1,no


In [61]:
X_.head(10)

,age,job,education,default,housing,loan,y
0,58,housemaid,basic.4y,unknown,yes,no,no
1,38,technician,professional.course,no,yes,no,no
2,81,retired,basic.4y,no,yes,no,no
3,49,blue-collar,basic.6y,no,yes,no,yes
4,38,unemployed,basic.4y,unknown,no,no,no
5,47,admin.,unknown,unknown,yes,no,no
6,33,admin.,university.degree,no,no,yes,no
7,31,technician,university.degree,no,yes,no,no
8,31,admin.,high.school,no,no,yes,no
9,51,housemaid,basic.9y,no,yes,no,no


特徴量データの上位10件を表示しました。

### Step3　特徴量データの前処理
識別器での学習を行う前に、特徴量を適切な形へ変換する必要があります。
その過程を見ていきましょう。

まず、欠損値を含むデータの扱いですが、本課題では、簡単のため単純に削除する手法を取ります。

In [62]:
df_temp = pd.concat([y_, X_], axis=1).dropna(how='any')# 欠損値を含むデータの削除
df_temp.index = range(len(df_temp)) 
y = df_temp.marital
X = df_temp[['age', 'housing', 'education', 'job']]

次に、housing 列は文字列であるため、数値に変換してあげる必要があります。ここでは、yes = 2, unknown = 1, no = 0 として変換します。

In [63]:
X.tail(10)

,age,housing,education,job
19990,33,yes,basic.9y,unemployed
19991,41,yes,university.degree,housemaid
19992,54,no,university.degree,technician
19993,44,yes,basic.9y,blue-collar
19994,59,no,basic.4y,retired
19995,24,no,university.degree,technician
19996,42,yes,unknown,technician
19997,31,no,professional.course,technician
19998,25,yes,professional.course,blue-collar
19999,33,no,high.school,technician


In [64]:
housing_le = LabelEncoder()
X.housing = housing_le.fit_transform(X.housing)
X.education = housing_le.fit_transform(X.education)
X.job = housing_le.fit_transform(X.job)

In [65]:
X.tail(10)

,age,housing,education,job
19990,33,2,2,10
19991,41,2,6,3
19992,54,0,6,9
19993,44,2,2,1
19994,59,0,0,5
19995,24,0,6,9
19996,42,2,7,9
19997,31,0,5,9
19998,25,2,5,1
19999,33,0,3,9


これで、数値データに置き換えられました。


最後に、それぞれの特徴量の数値の変動量に差による影響力の差を排除するため、正規化を行います。

In [66]:
mm_scaler = MinMaxScaler()

In [67]:
X.age = DataFrame(mm_scaler.fit_transform(DataFrame(X.age.copy())))
X.housing = DataFrame(mm_scaler.fit_transform(DataFrame(X.housing.copy())))

In [68]:
X.tail(10)

,age,housing,education,job
19990,0.197531,1.0,2,10
19991,0.296296,1.0,6,3
19992,0.456790,0.0,6,9
19993,0.333333,1.0,2,1
19994,0.518519,0.0,0,5
19995,0.086420,0.0,6,9
19996,0.308642,1.0,7,9
19997,0.172840,0.0,5,9
19998,0.098765,1.0,5,1
19999,0.197531,0.0,3,9


これで、正規化が出来ました。

### Step 4　識別器の実装
ロジスティック回帰の例を参考に、SVM、K-近傍法、決定木、ランダムフォレストを実装しましょう。

#### ロジスティック回帰
ロジスティック回帰の例を示します。実装にはscikit-learn(http://scikit-learn.org/stable/) の関数を使用します。

In [69]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1) # 訓練データとテストデータに分割
default_acc = {}

In [70]:
logistic_regression = LogisticRegression() # ロジスティック回帰モデルの作成
logistic_regression.fit(X_train, y_train) # 訓練データを使って学習
y_pred = logistic_regression.predict(X_test) # 予測値の取得

acc = accuracy_score(y_pred=y_pred, y_true=y_test)
print(f'accuracy:{acc}') # 精度の計算
default_acc['logistic_regression'] = acc

accuracy:0.657


上記の LogisticRegression( ) の部分を書き換えて、SVM、K-近傍法、決定木、ランダムフォレストを実装しましょう。

#### SVM
ここで、SVM (SVC) の実装を行ってください。特徴量も自由に選択して構いません。

In [71]:
svm = SVC()
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)

acc = accuracy_score(y_pred=y_pred, y_true=y_test)
print(f'accuracy:{acc}') # 精度の計算
default_acc['SVC'] = acc

accuracy:0.633


#### K-近傍法

ここで、K-近傍法 (KNeighborsClassifier) の実装を行ってください。特徴量も自由に選択して構いません。

In [72]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

acc = accuracy_score(y_pred=y_pred, y_true=y_test)
print(f'accuracy:{acc}') # 精度の計算
default_acc['KNN'] = acc

accuracy:0.6285


#### 決定木

ここで、決定木 (DecisionTreeClassifier) の実装を行ってください。特徴量も自由に選択して構いません。

In [73]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, y_train)
y_pred = decision_tree.predict(X_test)

acc = accuracy_score(y_pred=y_pred, y_true=y_test)
print(f'accuracy:{acc}') # 精度の計算
default_acc['decision_tree'] = acc

accuracy:0.6355


#### ランダムフォレスト

ここで、ランダムフォレスト (RandomForestClassifier) の実装を行ってください。特徴量も自由に選択して構いません。

In [74]:
random_forest = RandomForestClassifier()
random_forest.fit(X_train, y_train)
y_pred = random_forest.predict(X_test)

acc = accuracy_score(y_pred=y_pred, y_true=y_test)
print(f'accuracy:{acc}') # 精度の計算
default_acc['random_forest'] = acc

accuracy:0.64


#### Additional module: Bagging KNN classifier

In [75]:
from sklearn.ensemble import BaggingClassifier
bagging_knn = BaggingClassifier(KNeighborsClassifier(), max_samples=0.5, max_features=0.5)
bagging_knn.fit(X_train, y_train)
y_pred = bagging_knn.predict(X_test)

acc = accuracy_score(y_pred=y_pred, y_true=y_test)
print(f'accuracy:{acc}') # 精度の計算
default_acc['Bagging KNN'] = acc

accuracy:0.6325


#### Additional module: Naive Bayes

In [76]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_test)

acc = accuracy_score(y_pred=y_pred, y_true=y_test)
print(f'accuracy:{acc}') # 精度の計算
default_acc['naive_bayes'] = acc

accuracy:0.652


#### Additional module: XGBoost classifier

In [77]:
import xgboost
xgb = xgboost.XGBClassifier()
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)

acc = accuracy_score(y_pred=y_pred, y_true=y_test)
print(f'accuracy:{acc}') # 精度の計算
default_acc['xgboost'] = acc

accuracy:0.6705


#### Additional module: GBDT

In [78]:
from sklearn.ensemble import GradientBoostingClassifier
gbdt = GradientBoostingClassifier()
gbdt.fit(X_train, y_train)
y_pred = gbdt.predict(X_test)

acc = accuracy_score(y_pred=y_pred, y_true=y_test)
print(f'accuracy:{acc}') # 精度の計算
default_acc['GBDT'] = acc

accuracy:0.671


#### Additional module: Perceptron

In [79]:
from sklearn.linear_model import Perceptron
perceptron = Perceptron(tol=1e-3, random_state=RAND_SEED)
perceptron.fit(X_train, y_train)
y_pred = perceptron.predict(X_test)

acc = accuracy_score(y_pred=y_pred, y_true=y_test)
print(f'accuracy:{acc}') # 精度の計算
default_acc['Perceptron'] = acc

accuracy:0.59


#### Additional module: SGD

In [80]:
from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier(max_iter=1000, tol=1e-3)
sgd.fit(X_train, y_train)
y_pred = sgd.predict(X_test)

acc = accuracy_score(y_pred=y_pred, y_true=y_test)
print(f'accuracy:{acc}') # 精度の計算
default_acc['SGD'] = acc

accuracy:0.6435


#### Additional module: AdaBoost

In [81]:
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier(n_estimators=100, random_state=RAND_SEED)
ada.fit(X_train, y_train)
y_pred = ada.predict(X_test)

acc = accuracy_score(y_pred=y_pred, y_true=y_test)
print(f'accuracy:{acc}') # 精度の計算
default_acc['AdaBoost'] = acc

accuracy:0.6695


#### Additional module: Voting  

In [82]:
from sklearn.ensemble import VotingClassifier
voting = VotingClassifier(estimators={
    ('gbdt', gbdt),
    ('random_forest', random_forest),
    ('logistic_regression', logistic_regression),
    #('nb', nb),
    ('bagging_knn', bagging_knn),
    ('decision_tree', decision_tree),
    ('knn', knn),
    ('xgb', xgb),
    ('ada', ada),
    #('svm', svm),
    #('perceptron', perceptron),
    #('sgd', sgd),
}, voting='soft')
voting.fit(X_train, y_train)
y_pred = voting.predict(X_test)

acc = accuracy_score(y_pred=y_pred, y_true=y_test)
print(f'accuracy:{acc}') # 精度の計算
default_acc['Voting'] = acc
print("Accurate for default classifiers:")
default_acc

accuracy:0.67
Accurate for default classifiers:


{'logistic_regression': 0.657,
 'SVC': 0.633,
 'KNN': 0.6285,
 'decision_tree': 0.6355,
 'random_forest': 0.64,
 'Bagging KNN': 0.6325,
 'naive_bayes': 0.652,
 'xgboost': 0.6705,
 'GBDT': 0.671,
 'Perceptron': 0.59,
 'SGD': 0.6435,
 'AdaBoost': 0.6695,
 'Voting': 0.67}

# 課題２: モデルの改善を行おう！
課題1で作成した各モデルについて、どうすればさらに良い精度を出すことができるのか、パラメータの調整や特徴量の選定などを行いましょう。
最後に、この課題において最善と考える特徴量とモデルを述べ、その理由も示してください。

まず、選定した特徴量を用いて、課題１で行ったStep1〜Step3 を行いましょう。ただし、Step 2〜Step 3の処理は、preprocessingという関数で定義してください。ここで定義した関数を用いて評価を行います。
引数には、読み込んだデータを指定してください。返り値には、前処理を行った後の選定した特徴量 ( X )とmarital ( y )を指定してください。

In [83]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.impute import SimpleImputer

In [84]:
### Step 1　　データの読み込み
#raw_data.dropna(how='any')
raw_data = pd.read_csv('train.csv')
train_data, test_data = train_test_split(raw_data, test_size=0.1, random_state=RAND_SEED)
num_attrs = ["age"]
cat_attrs = ["job", "education", "default", "housing", "loan", "y"]
feature_attrs = num_attrs + cat_attrs
target = ["marital"]

num_pipeline = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    #StandardScaler(),
    MinMaxScaler(),
)

preproc_onehot = make_column_transformer(
    (num_pipeline, num_attrs),
    (OneHotEncoder(), cat_attrs),
)

preproc_ordinal = make_column_transformer(
    (num_pipeline, num_attrs),
    (OrdinalEncoder(), cat_attrs),
)

full_pipeline = make_pipeline(
    preproc_onehot,
    ada,
    #PolynomialFeatures(degree=2),
    #GaussianNB(),
    #xgboost.XGBClassifier(random_state=RAND_SEED),
    #LogisticRegression(random_state=RAND_SEED),
    #SVC(),
    #GradientBoostingClassifier()
    #voting
    #DecisionTreeClassifier(),
    #KNeighborsClassifier(),
)

full_pipeline.fit(train_data[feature_attrs], train_data[target])
print(f"acc: {accuracy_score(full_pipeline.predict(test_data[feature_attrs]), test_data[target]):.4f}")

acc: 0.6710


In [85]:
### 課題１で行ったStep 2〜Step 3を関数として定義してください。
def preprocessing(df_train_data):
    ### Step 2　　使用する列の抽出
    y_ = df_train_data.marital# ラベルデータ
    X_ = df_train_data[['age', 'housing']]
    
    ### Step 3　　特徴量データの前処理
    # 欠損値を含むデータの削除
    df_temp = pd.concat([y_, X_], axis=1).dropna(how='any')
    df_temp.index = range(len(df_temp)) 
    y = df_temp.marital
    X = df_temp[['age', 'housing']]
    
    # 文字列を数値データへ変換
    housing_le = LabelEncoder()
    X.housing = housing_le.fit_transform(X.housing)
    
    # 数値データの正規化
    mm_scaler = MinMaxScaler()
    X.age = DataFrame(mm_scaler.fit_transform(DataFrame(X.age.copy())))
    X.housing = DataFrame(mm_scaler.fit_transform(DataFrame(X.housing.copy())))
    
    return X, y

上記で定義した関数を実行します。

In [86]:
X, y = preprocessing(df_train_data)

次に、この課題における最善なモデルを構築しましょう。課題１で使用したモデル以外を使用しても構いません。
下記は、ロジスティック回帰の場合の例です。

In [87]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1) # 訓練データとテストデータに分割

In [88]:
logistic_regression = LogisticRegression() # ロジスティック回帰モデルの作成
logistic_regression.fit(X_train, y_train) # 訓練データを使って学習
y_pred = logistic_regression.predict(X_test) # 予測値の取得
print('accuracy:{}'.format(accuracy_score(y_pred=y_pred, y_true=y_test))) # 精度の計算

accuracy:0.67


### 考察１
この課題において最善と考える特徴量を述べ、その理由も示してください。

（ここに記入してください）

### 考察２
この課題において最善と考えるモデルを述べ、その理由も示してください。

（ここに記入してください）

最後に、以下の best_model に構築したモデルを記入してください。この best_model を使用して精度評価を行います。

In [89]:
best_model = logistic_regression     # この例では、ロジスティック回帰を選択したので、logistic_regression を記入しています。
pickle.dump(best_model, open('best_model.pkl','wb'))

### --------評価用--------
ここから下は評価用となります。上の欄にモデルを記入した後は、配布資料に書いてある手順に沿ってファイルを提出してください。

In [90]:
# df_test_data = pd.read_csv('bank_test.csv')
# preprocessing()
# X_test = X
# y_test = y

In [91]:
# best2_model = pickle.load(open('best_model.pkl', 'rb'))
# y_pred =best2_model.predict(X_test)
# print('accuracy:{}'.format(accuracy_score(y_pred=y_pred, y_true=y_test)))